# Basic Configs

In [1]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


## Imports

In [2]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

from torchvision import datasets, transforms
from torch.utils import data

import numpy as np
import matplotlib.pyplot as plt

## Global Variables

In [3]:
cuda = torch.cuda.is_available()
device = torch.device("cuda" if cuda else "cpu")
cpu = torch.device("cpu")
batch_size = 128
num_workers = 4

## Load Repo
- Rename the folder to `project`

In [ ]:
# !rm -rf project

In [4]:
!git clone https://github.com/effie-0/IDL-Project.git

Cloning into 'IDL-Project'...
remote: Enumerating objects: 253, done.
remote: Counting objects: 100% (253/253), done.
remote: Compressing objects: 100% (187/187), done.
remote: Total 253 (delta 132), reused 132 (delta 54), pack-reused 0
Receiving objects: 100% (253/253), 1.60 MiB | 1.67 MiB/s, done.
Resolving deltas: 100% (132/132), done.


In [5]:
!mv IDL-Project project

In [6]:
!cd project; git checkout attack

Branch 'attack' set up to track remote branch 'attack' from 'origin'.
Switched to a new branch 'attack'


# Load Functions

In [7]:
from project.summarize.load_data import AdvDataset, get_data

## Load Dataset

In [8]:
trainset, trainloader, testset, testloader = get_data(batch_size)

Extracting ./data/cifar-10-python.tar.gz to ./data
Files already downloaded and verified


## Load Models

In [9]:
from project.summarize.classifier import ResidualBlock, ResNet18, cifar_resnet20, ResNet50, VGG

In [10]:
resnet18 = ResNet18(ResidualBlock)
resnet20 = cifar_resnet20('cifar10')  # With pretrained model
resnet50 = ResNet50()
vgg16 = VGG('VGG16')

Downloading: "https://github.com/chenyaofo/CIFAR-pretrained-models/releases/download/resnet/cifar10-resnet20-30abc31d.pth" to /root/.cache/torch/hub/checkpoints/cifar10-resnet20-30abc31d.pth


### Load Trained Models

In [11]:
vgg16_path = "/content/gdrive/My Drive/18-786 IDL/IDL.dll/models/vgg16.pth"
vgg16.load_state_dict(torch.load(vgg16_path)['model_state_dict'])

resnet18_path = "/content/gdrive/My Drive/18-786 IDL/IDL.dll/models/ResNet18.pth"
resnet18.load_state_dict(torch.load(resnet18_path)['model_state_dict'])

resnet50_path = "/content/gdrive/My Drive/18-786 IDL/IDL.dll/models/Copy of ResNet50_49.pth"
resnet50.load_state_dict(torch.load(resnet50_path)['model_state_dict'])

<All keys matched successfully>

In [12]:
models = {
    'vgg16': vgg16,
    'resnet18': resnet18,
    'resnet20': resnet20,
    'resnet50': resnet50,
}

# Generate the Adversarial Samples

In [13]:
criterion = nn.CrossEntropyLoss()

## Evaluate the models

In [14]:
from project.summarize.normal_train import evaluate
from project.summarize.attack import get_grad

## Eps = 0.03

In [15]:
def get_avg_grad(models, criterion, data, target):
    # Set requires_grad attribute of tensor. Important for Attack
    data.requires_grad = True

    w = len(models)

    # Forward pass the data through the model
    output = 0.0
    for model in models:
        output += model(data) / w
    init_pred = output.max(1, keepdim=True)[1] # get the index of the max log-probability

    # mask: 1 for correct, only update grad on correct image
    mask = torch.eq(init_pred.flatten(), target.flatten()).float()

    # Calculate the loss
    loss = criterion(output, target)

    # Zero all existing gradients
    model.zero_grad()

    # Calculate gradients of model in backward pass
    loss.backward()

    # Collect datagrad
    return data.grad.data, mask

In [16]:
def fgsm_attack(image, epsilon, models, criterion, target, decay_rate=1.0):
    rounds = 1
    alpha = epsilon / rounds
    grad = 0.0
    x = image.detach().clone()
    w = len(models)
    data_grad = 0.0
    mask = torch.zeros((len(image))).view(-1, 1, 1, 1).to(device)
    for t in range(int(rounds)):
        # for model in models:
        #     single_data_grad, single_mask = get_grad(model, criterion, x, target)
        #     data_grad += single_data_grad / w
        #     mask = torch.logical_or(mask, single_mask.view(-1, 1, 1, 1))
        data_grad, mask = get_avg_grad(models, criterion, x, target)
        grad = decay_rate * grad + data_grad / torch.norm(data_grad, p=1)
        if mask is None:
            sign_data_grad = data_grad.sign()
        else:
            # Collect the element-wise sign of the data gradient
            sign_data_grad = torch.mul(data_grad.sign(), mask.view(-1, 1, 1, 1))
        x = x + alpha * sign_data_grad
        x = x.detach().clone()
    return x


In [17]:
def generate_attack(test_loader, models, eps, criterion):
    adv_examples = []
    # Loop over all examples in test set
    for image, target in test_loader:
        # Send the data and label to the device
        image, target = image.to(device), target.to(device)

        # Call Attack
        adv_data = fgsm_attack(image, eps, models, criterion, target)

        adv_data = adv_data.squeeze().detach().cpu().numpy()
        adv_examples.append( (target.flatten().detach().cpu().numpy(), adv_data) )
        # pred_list.append((init_pred.flatten().detach().cpu().numpy(), final_pred.flatten().detach().cpu().numpy()))
    
    label = [j for i in adv_examples for j in i[0]]
    adv_ex = [j for i in adv_examples for j in i[1]]

    dataset = AdvDataset(adv_ex, label)
    loader = data.DataLoader(dataset,
                             batch_size=batch_size, 
                             shuffle=False,
                             num_workers=num_workers)

    # Return the accuracy and an adversarial example
    return dataset, loader

In [18]:
other_samples = {}
eps = 0.03

for model_name, model in models.items():
    # model.to(device)
    # model.eval()
    tmp_models = models.copy()
    del tmp_models[model_name]
    tmp_models = tmp_models.values()
    for m in tmp_models:
        m.to(device)
        m.eval()
    print(len(tmp_models))
    dataset, loader = generate_attack(testloader, tmp_models, eps, criterion)
    # model.to(cpu)
    for m in tmp_models:
        m.to(cpu)
    torch.cuda.empty_cache()
    other_samples[model_name] = (dataset, loader)

3
3
3
3


# 4x4 Matrix

In [19]:
matrix = {}
for model_name, model in models.items():
    line = {}
    matrix[model_name] = line
    model.to(device)
    for adv_name, (dataset, loader) in other_samples.items():
        print('classifier: ', model_name, ';  adversarial data does not come from: ', adv_name)
        loss, acc = evaluate(model, loader, criterion, device)
        line[adv_name] = acc
        print('acc = ', acc)
    model.to(cpu)
    torch.cuda.empty_cache()

classifier:  vgg16 ;  adversarial data does not come from:  vgg16
acc =  81.67999999999999
classifier:  vgg16 ;  adversarial data does not come from:  resnet18
acc =  66.5
classifier:  vgg16 ;  adversarial data does not come from:  resnet20
acc =  56.84
classifier:  vgg16 ;  adversarial data does not come from:  resnet50
acc =  65.92
classifier:  resnet18 ;  adversarial data does not come from:  vgg16
acc =  72.25
classifier:  resnet18 ;  adversarial data does not come from:  resnet18
acc =  81.47999999999999
classifier:  resnet18 ;  adversarial data does not come from:  resnet20
acc =  61.73
classifier:  resnet18 ;  adversarial data does not come from:  resnet50
acc =  72.83
classifier:  resnet20 ;  adversarial data does not come from:  vgg16
acc =  32.519999999999996
classifier:  resnet20 ;  adversarial data does not come from:  resnet18
acc =  32.300000000000004
classifier:  resnet20 ;  adversarial data does not come from:  resnet20
acc =  71.05
classifier:  resnet20 ;  adversarial 